<a href="https://colab.research.google.com/github/Zarawar5555/OpenCvLightControl/blob/main/OpwnCV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:

import matplotlib
matplotlib.use('Agg')
import cv2
import mediapipe as mp
import pyfirmata2
import time
from mediapipe.tasks.python.audio import audio_classifier

arduino_port = "COM8"
board = pyfirmata2.Arduino(arduino_port)
LED_1 = 3
LED_2 = 5
LED_3 = 6
LED_4 = 7
LED_5 = 8

led_pins = [LED_1, LED_2, LED_3, LED_4, LED_5]
for pin in led_pins:
    board.digital[pin].mode = pyfirmata2.OUTPUT
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1)
mp_draw = mp.solutions.drawing_utils
cap = cv2.VideoCapture(0)

def count_fingers(hand_landmarks):
    fingers_up = 0


    if hand_landmarks.landmark[4].x > hand_landmarks.landmark[3].x:
        fingers_up += 1
    finger_tips = [8, 12, 16, 20]
    for tip in finger_tips:
        if hand_landmarks.landmark[tip].y < hand_landmarks.landmark[tip - 2].y:
            fingers_up += 1

    return fingers_up

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frame = cv2.flip(frame, 1)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    result = hands.process(rgb_frame)

    fingers = 0
    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            fingers = count_fingers(hand_landmarks)

    print(f"Fingers detected: {fingers}")


    led_states = [0, 0, 0, 0, 0]

    if fingers == 1:
        led_states = [1, 0, 0, 0, 0]
    elif fingers == 2:
        led_states = [1, 1, 0, 0, 0]
    elif fingers == 3:
        led_states = [1, 1, 1, 0, 0]
    elif fingers == 4:
        led_states = [1, 1, 1, 1, 0]
    elif fingers == 5:
        led_states = [1, 1, 1, 1, 1]
    for pin, state in zip(led_pins, led_states):
        board.digital[pin].write(state)

    time.sleep(0.05)
    cv2.imshow("Hand Gesture LED Control", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()
board.exit()


SerialException: [Errno 2] could not open port COM8: [Errno 2] No such file or directory: 'COM8'